In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score, roc_auc_score

pd.set_option("display.max_columns", 50)

In [2]:
df = pd.read_csv("../data/processed/moneypuck_features_final.csv")
df.head()

,team,season,name,gameId,playerTeam,opposingTeam,home_or_away,gameDate,position,situation,xGoalsPercentage,corsiPercentage,fenwickPercentage,iceTime,xOnGoalFor,xGoalsFor,xReboundsFor,xFreezeFor,xPlayStoppedFor,xPlayContinuedInZoneFor,xPlayContinuedOutsideZoneFor,flurryAdjustedxGoalsFor,scoreVenueAdjustedxGoalsFor,flurryScoreVenueAdjustedxGoalsFor,shotsOnGoalFor,...,highDangerShotsAgainst,lowDangerxGoalsAgainst,mediumDangerxGoalsAgainst,highDangerxGoalsAgainst,lowDangerGoalsAgainst,mediumDangerGoalsAgainst,highDangerGoalsAgainst,scoreAdjustedShotsAttemptsAgainst,unblockedShotAttemptsAgainst,scoreAdjustedUnblockedShotAttemptsAgainst,dZoneGiveawaysAgainst,xGoalsFromxReboundsOfShotsAgainst,xGoalsFromActualReboundsOfShotsAgainst,reboundxGoalsAgainst,totalShotCreditAgainst,scoreAdjustedTotalShotCreditAgainst,scoreFlurryAdjustedTotalShotCreditAgainst,playoffGame,days_rest,back_to_back,rolling_goals_5,rolling_xg_5,fatigue_penalty,fatigue_adj_goals,fatigue_adj_xg
0,ANA,2017,ANA,2017020011,ANA,ARI,HOME,2017-10-05,Team Level,all,0.4752,0.5271,0.5700,3600.0,40.890,3.632,2.791,9.510,1.307,22.583,17.177,3.533,3.461,3.364,41.0,...,5.0,0.800,1.087,2.123,1.0,1.0,2.0,64.860,43.0,45.608,10.0,0.518,1.251,1.251,3.278,3.430,3.386,0,3.0,0,NaN,NaN,1.0,NaN,NaN
1,ANA,2017,ANA,2017020029,ANA,PHI,HOME,2017-10-07,Team Level,all,0.3855,0.4579,0.4625,3644.0,25.656,1.617,1.630,5.922,0.859,14.976,11.997,1.608,1.548,1.540,23.0,...,4.0,0.843,0.475,1.260,2.0,1.0,0.0,59.958,43.0,44.544,2.0,0.481,0.533,0.533,2.526,2.609,2.436,0,1.0,0,NaN,NaN,1.0,NaN,NaN
2,ANA,2017,ANA,2017020038,ANA,CGY,HOME,2017-10-09,Team Level,all,0.5044,0.5493,0.5688,3600.0,41.493,3.616,3.125,9.367,1.565,25.928,18.400,3.499,3.511,3.396,43.0,...,4.0,0.805,0.885,1.863,0.0,1.0,1.0,67.842,47.0,49.699,6.0,0.485,0.000,0.000,4.038,4.212,3.999,0,1.0,0,NaN,NaN,1.0,NaN,NaN
3,ANA,2017,ANA,2017020049,ANA,NYI,HOME,2017-10-11,Team Level,all,0.5105,0.4403,0.4700,3600.0,32.193,2.640,2.238,8.067,1.109,18.420,14.527,2.584,2.621,2.566,30.0,...,1.0,1.330,0.958,0.243,1.0,0.0,1.0,74.427,53.0,52.362,9.0,0.575,0.469,0.469,2.638,2.678,2.645,0,1.0,0,NaN,NaN,1.0,NaN,NaN
4,ANA,2017,ANA,2017020060,ANA,COL,AWAY,2017-10-13,Team Level,all,0.1913,0.3197,0.3372,3600.0,18.380,0.947,1.141,4.280,0.651,11.013,10.968,0.943,0.964,0.960,18.0,...,3.0,0.994,1.703,1.305,2.0,0.0,1.0,83.030,57.0,57.309,1.0,0.663,0.310,0.310,4.356,4.329,4.260,0,1.0,0,NaN,NaN,1.0,NaN,NaN


In [3]:
features = [
    "days_rest",
    "back_to_back",
    "rolling_goals_5",
    "rolling_xg_5",
    "fatigue_adj_goals",
    "fatigue_adj_xg"
]

X = df[features]
y = df["goalsFor"]

In [4]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [6]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

reg_pipeline = Pipeline([
    ("imputer", SimpleImputer(strategy="mean")),
    ("model", LinearRegression())
])

In [7]:
reg_pipeline.fit(X_train, y_train)

,steps,"[('imputer', ...), ('model', ...)]"
,transform_input,None
,memory,None
,verbose,False
,missing_values,nan
,strategy,'mean'
,fill_value,None
,copy,True
,add_indicator,False
,keep_empty_features,False
,fit_intercept,True


In [8]:
y_pred = reg_pipeline.predict(X_test)

rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)

rmse, r2

(np.float64(1.7330926815477532), 0.010785205183687752)

In [10]:
df["win"] = (df["goalsFor"] > df["goalsAgainst"]).astype(int)

In [11]:
X_train, X_test, y_train, y_test = train_test_split(
    X, df["win"], test_size=0.2, random_state=42
)

In [13]:
clf_pipeline = Pipeline([
    ("imputer", SimpleImputer(strategy="mean")),
    ("model", LogisticRegression(max_iter=1000))
])

In [14]:
clf_pipeline.fit(X_train, y_train)

win_pred = clf_pipeline.predict(X_test)
win_proba = clf_pipeline.predict_proba(X_test)[:, 1]

accuracy = accuracy_score(y_test, win_pred)
auc = roc_auc_score(y_test, win_proba)

accuracy, auc

(0.553575508339045, 0.5553529369855665)